In [2]:
import pandas as pd
from datetime import datetime

In [61]:
df = pd.DataFrame()
for i in range(6):
    df = pd.concat([df,pd.read_excel(f'../raw/Plakate_1945-90_ImportCBS_Teil{i+1}.xlsx', dtype={'ERJ':'str'})], ignore_index=True)
df.loc[df.Sprache == 'ger;hun', 'Sprache'] = 'ger/1hun'
df.loc[df.IMD == ' T', 'IMD'] = 'T'
df.loc[df.ERJ == '1950-60?', 'ERJ'] = '1950-1960?'
# df.info()

In [115]:
df[(df.Signatur == '1945-90/2871')]

,IMD,ERJ,Signatur,Format,Anzahl,Form,Anmerkung,Internvermerk,Trägermaterial,Zustand,kons_bemerkung,Altsignatur,Ländercode,Sprache,Erscheinungsort
2893,T/B,1946?,1945-90/2871,D,1 ungezähltes Blatt,NaN,NaN,NaN,NaN,NaN,NaN,78/53/P152(132),XA-DE,ger,Sowjetische Besatzungszone


In [116]:
# df[pd.isna(df['ERJ'])].to_excel('../raw/Plakate_1945-90_ERJ-fehlt.xlsx', index=False)
# df.Signatur.value_counts()
# df[df.Erscheinungsort == '[Ort nicht ermittelbar]']
# df[pd.isna(df.Erscheinungsort) | (df.Erscheinungsort == '[Ort nicht ermittelbar]') ]
# df[(df.Signatur == '1945-90/982') | (df.Signatur == '1945-90/13563') | (df.Signatur == '1945-90/914')].to_excel('../raw/Plakate_1945-90_Signatur-doppelt.xlsx', index=False)
signaturliste = [int(x.split('-')[0]) for x in df.Signatur.str[8:].to_list()]
signaturliste.sort()
for element in range(len(signaturliste)):
    if element > 0:
        if signaturliste[element-1] < signaturliste[element]-1:
            print(signaturliste[element-1],signaturliste[element])

    


3713 3718
4082 4084


In [79]:
def datensatz_schreiben(row):
    pica = "\n0500 Pa\n"

    if row.IMD == 'T':
        pica += "0501 Text$btxt\n"
    elif row.IMD == 'T/B':
        pica += "0501 Text$btxt\n0501 unbewegtes Bild$bsti\n"
    elif row.IMD == 'B':
        pica += "0501 unbewegtes Bild$bsti\n"
    
    pica += f"""0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy\n"""

    if len(str(row.ERJ)) == 4:
        # nur ein datum
        pica += f"1100 {row.ERJ}\n1110 *{row.ERJ}$4ezth\n"
    elif len(str(row.ERJ)) == 5 and str(row.ERJ.endswith('?')) == 'True':
        # ein jahr mit fragezeichen
        pica += f"1100 {row.ERJ[:4]}$n[{row.ERJ}]\n1110 {row.ERJ}*{row.ERJ[:4]}$4ezth\n"
    elif len(str(row.ERJ)) == 9:
        # zwei daten
        pica += f"1100 {row.ERJ[:4]}$n[zwischen {row.ERJ[:4]} und {row.ERJ[5:]}]\n1110 zwischen {row.ERJ[:4]} und {row.ERJ[5:]}*{row.ERJ}$a{row.ERJ[:4]}$b{row.ERJ[5:]}$4ezth\n"
    elif len(str(row.ERJ)) == 10:
        # zwei daten mit fragezeichen
        pica += f"1100 {row.ERJ[:4]}$n[zwischen {row.ERJ[:4]} und {row.ERJ[5:]}]\n1110 zwischen {row.ERJ[:4]} und {row.ERJ[5:]}*{row.ERJ[:9]}$a{row.ERJ[:4]}$b{row.ERJ[5:9]}$4ezth\n"
    else:
        pica += f"\n\n1100 FEHLER {row.Signatur}\n\n"
    
    if pd.isna(row.Trägermaterial):
        pica += "1130 TB-papier\n1130 !040445224!\n"
    else:
        pica += f"1130 {row.Trägermaterial}\n"
    
    if pd.isna(row.Form):
        pica += "1131 !04046198X!\n"
    else:
        pica += f"1131 {row.Form}\n"
    
    pica += "1132 a1-analog;a2-druck;f2-blatt\n"

    if pd.notna(row.Sprache):
        pica += f"1500 /1{row.Sprache}\n"
    
    pica += f"""1700 /1{row.Ländercode}
2105 04,P01
4000 Plakat Nr. {row.Signatur} der Plakatsammlung 1945-1990
4019 Plakat Nr. {row.Signatur} der Plakatsammlung 1945-1990\n"""
    
    if pd.notna(row.Erscheinungsort) and row.Erscheinungsort != '[Ort nicht ermittelbar]':
        pica += f"4030 [{row.Erscheinungsort}] : [Verlag nicht ermittelbar]"
    elif row.Erscheinungsort == '[Ort nicht ermittelbar]' or pd.isna(row.Erscheinungsort):
        pica += f"4030 [Ort nicht ermittelbar] : [Verlag nicht ermittelbar]"
    
    pica += f"""
4060 {row.Anzahl}
4062 Format {row.Format}
4105 !1238824013!\n"""
    
    if pd.notna(row.Anmerkung):
        pica += f"4201 {row.Anmerkung}\n"
    
    pica += f"4276 nachträglich durch Auswertung der 4201\n"
    
    if pd.notna(row.Internvermerk):
        pica += f"4700 |BSM|{row.Internvermerk}\n"
       
    
    
    pica += f"""5590 [Objektgattung]
5590 !041133579!
7001 {now.strftime('%d-%m-%y')} : x
7100 GS PL {row.Signatur} @ m\n"""
    be_4801 = list()
    for element in [(row.Zustand), (row.kons_bemerkung)]:
        if type(element) == str:
            be_4801.append(element)
        elif type(element) == float:
            pass
        else:
            be_4801.append(f'FEHLER in {row.Signatur}')
    
    if len(be_4801) > 0:
        pica += f"4801 {'; '.join(be_4801)}\n"
    
    pica += """6800 [Provenienz]
6800 !004709608! *1961-1993
6800 Stempel
7109 !!DBSM/S!! ; GS PL {row.Signatur}
8034 Dieses Plakat ist Teil eines Erschließungsprojekts und derzeit nicht ausleihbar.\n"""
    if pd.notna(row.Altsignatur):
        pica += f"8598 {row.Altsignatur}\n"
    return pica

In [80]:
now = datetime.now()
with open(f"../dat/plakate1945-90-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/plakate1945-90-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(datensatz_schreiben(row))
            f2.write(datensatz_schreiben(row))

with open(f"../dat/plakate1945-90-recent-sample.dat", 'w') as f:
    for row in df.sample(5).itertuples():
        f.write(datensatz_schreiben(row))

# bekannte Fehler
- ERJ hat manchmal keinen Fehler, was wird standardwert für 1100 und 1110?
- 0500 und 0501 noch nicht geklärt
- 4276